# NLP Final Project
Sam Ding

In [372]:
# basic data analytics
import pandas as pd
import numpy as np
import sklearn
import pickle

# nlp modules
import nltk
import spacy
import re

import multiprocessing
import string

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer


import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings

# warnings.simplefilter('once')
warnings.simplefilter('ignore')

num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors-1

print(f'Using {workers} workers')

Using 7 workers


In [373]:
# %%time

# df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
# df_news_final_project.shape

In [374]:
# # zero-shot classification
# import torch
# from transformers import pipeline
# classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [375]:
df_news_final_project = pd.read_csv('sample_600.csv', index_col=0)
# df_news_final_project.shape

In [376]:
df_news_final_project.head(3)

,url,date,language,title,text
39396,https://www.wkms.org/npr-news/npr-news/2022-10...,2022-10-10,en,Artificial intelligence could soon diagnose il...,\n\nArtificial intelligence could soon diagnos...
143316,https://www.wbko.com/prnewswire/2022/08/25/ult...,2022-08-25,en,UltraSight Receives CE Mark for Novel Cardiac ...,UltraSight Receives CE Mark for Novel Cardiac ...
100092,https://www.marketscreener.com/quote/stock/POO...,2022-11-08,en,"IN BRIEF: Poolbeg makes ""significant breakthro...","\n\nIN BRIEF: Poolbeg makes ""significant break..."


In [377]:
# Clean-up newlines
# df_news_final_project['text_clean'] = df_news_final_project['text'].str.replace('\n', ' ')

# clean up tabs
df_news_final_project['text_clean'] = df_news_final_project['text'].str.replace('\t', ' ')

# clean up links
df_news_final_project['text_clean'] = df_news_final_project['text_clean'].str.replace(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '')

# clean up remnants of web crawls
df_news_final_project['text_clean'] = df_news_final_project['text_clean'].str.replace(r'&#\d+;', '', case=False)

In [378]:
# NER
nlp = spacy.load("en_core_web_md")

indexlist = []
entities = []
labels = []

docs = nlp.pipe(
    df_news_final_project['title'].tolist(),
    disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"],
    batch_size=200,
    n_process=2
)

for i, doc in enumerate(docs):
    index = df_news_final_project.index[i]
    for ent in doc.ents:
        indexlist.append(index)
        entities.append(ent.text)
        labels.append(ent.label_)


ner_df = pd.DataFrame({"Index": indexlist, "Entities":entities,'Labels':labels})

In [379]:
ner_df

,Index,Entities,Labels
0,143316,UltraSight,ORG
1,143316,CE Mark,PERSON
2,102690,Square Peg,PERSON
3,102690,AI,ORG
4,21501,1.04,CARDINAL
...,...,...,...
1333,9710,Artificial Intelligence and Machine Learning,ORG
1334,45140,Hexaware Collaborate to,ORG
1335,45140,Help Customers Accelerate,ORG
1336,45140,Journey,PRODUCT


In [380]:
# find indexes where labels have no ORG

with_org_list = list(ner_df[ner_df['Labels'] == 'ORG']['Index'].unique())

no_org_list = list(set(df_news_final_project.index) - set(with_org_list))

# get the title for those indexes

df_news_final_project[df_news_final_project.index.isin(no_org_list)]['title']

39396     Artificial intelligence could soon diagnose il...
100092    IN BRIEF: Poolbeg makes "significant breakthro...
21501     Patch 1.04: A Wagonload of AI · Grand Tacticia...
83882     Mohammad Hosseini: Should we bring AI into hos...
151392    SHUTTERSTOCK PARTNERS WITH OPENAI AND LEADS TH...
                                ...                        
9820          Artificial Intelligence Wish List - NewsBreak
119656    Rockies' trade deadline: Trevor Story, Jon Gra...
129684    Can ChatGPT help with investments if you want ...
117860    BuzzFeed to use artificial intelligence for co...
150157    How the KU community feels about ChatGPT and w...
Name: title, Length: 171, dtype: object

In [381]:
df_news_final_project.head(2)

,url,date,language,title,text,text_clean
39396,https://www.wkms.org/npr-news/npr-news/2022-10...,2022-10-10,en,Artificial intelligence could soon diagnose il...,\n\nArtificial intelligence could soon diagnos...,\n\nArtificial intelligence could soon diagnos...
143316,https://www.wbko.com/prnewswire/2022/08/25/ult...,2022-08-25,en,UltraSight Receives CE Mark for Novel Cardiac ...,UltraSight Receives CE Mark for Novel Cardiac ...,UltraSight Receives CE Mark for Novel Cardiac ...


In [382]:
notitle = []
for i in df_news_final_project.index.values:
    notitle.append(df_news_final_project['text_clean'][i].replace(df_news_final_project['title'][i], 'Hahahah 23, 2026')) # replace by this chunk so title can also be split by pattern

df_news_final_project['text_notitle'] = notitle

# drop everything after string 'for more information'ABC

df_news_final_project['text_notitle'] = df_news_final_project['text_notitle'].str.split(r'[F|f]or more information', expand=True)[0]


In [383]:
pattern = r'(\w{3,10}\.*\s\d{1,2}\,*\s20\d{2})|(\d{1,2}\s\w{3,10}\.*\s20\d{2})|\n+'

df_news_final_project['split'] = df_news_final_project['text_notitle'].apply(lambda x: re.split(pattern=pattern, string=x))
df_news_final_project['split_len'] = df_news_final_project['split'].apply(lambda x: len(x))

len 1 was 116, after new pattern 78, after new pattern all splitable.

In [384]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_most_similar(index):
    '''
    This function takes in an index of the dataframe and returns the most similar text to the title,
    filtering out other unnecessary texts.
    '''

    title = df_news_final_project['title'][index]
    texts = df_news_final_project['split'][index]

    # print('Original:\n', texts)

    # compare capital letters and periods, drop if there are more capital letters in the texts
    texts = [x for x in texts if x != None]

    # print('Dropping None:\n', texts)

    # print('Dropping more period than capital:\n', texts)

    # take out texts that are too short
    texts = [x for x in texts if len(x) > 150]

    # print('Dropping too short:\n', texts)

    # drop texts with Tab patterns
    pattern = r'([A-Z][a-z]+(\n|\t)+){4}'
    texts =  [re.sub(pattern, '', x) for x in texts]

    # print('Dropping Tab patterns:\n', texts)

    # keep texts with small letter patterns
    pattern = r'([a-z]+\s){4}'
    texts = [x for x in texts if re.search(pattern, x) != None]

    # print('Keeping small letter patterns:\n', texts)


    # return NA if there are no text splits left
    if len(texts) == 0:
        return np.nan

    # Tokenize the title and texts
    tokenized_title = nltk.word_tokenize(title.lower())
    tokenized_texts = [nltk.word_tokenize(text.lower()) for text in texts]

    # Convert the tokenized texts to strings
    text_strings = [' '.join(tokens) for tokens in tokenized_texts]

    # print('Tokenized texts:\n', len(text_strings))

    index_to_return = 0

    # Create a TF-IDF vectorizer and fit it to the text strings
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(text_strings)
        # print(tfidf_matrix.shape)

    # Compute the cosine similarity between the title and each text
    title_vector = vectorizer.transform([' '.join(tokenized_title)])
    similarity_scores = cosine_similarity(title_vector, tfidf_matrix)

    most_similar_index = similarity_scores.argmax()
        
    most_similar_score = similarity_scores.max()
    index_to_return = most_similar_index
        
    list_articles = []
    list_scores = []

    for i in range(0,len(similarity_scores[0])):
        if similarity_scores[0][i] >= 0.07:
            list_articles.append(text_strings[i])
            list_scores.append(similarity_scores[0][i])
        # print(len(text_strings), len(similarity_scores[0]))
                
        # if len(text_strings) > 1:
        #     second_most_similar_index = similarity_scores.argsort()[0][-2]
        #     # print(similarity_scores)
        #     second_most_similar_score = similarity_scores[0][second_most_similar_index]
        #     if most_similar_score > 0.2:
        #         index_to_return = second_most_similar_index
        # # return texts[index_to_return]
        # return 

    # except:
    #     pass
    

    return ' '.join(list_articles)
    


In [385]:
most_similar = []

for i in df_news_final_project.index.values:
    most_similar.append(get_most_similar(i))

df_news_final_project['text_clean'] = most_similar

df_news_final_project.dropna(subset=['text_clean'], inplace=True)

df_news_final_project.shape

(593, 9)

In [386]:
from pyspark.sql.functions import udf
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import StopWordsCleaner, Tokenizer
import sparknlp

# start Spark session
spark = sparknlp.start()

# create Spark dataframe
df_spark = spark.createDataFrame(df_news_final_project)

# define a UDF that detects if the article has keywords
keywords = ['artificial intelligence', 'machine learning', 'data science', 'data analytics']

def has_keywords(text):
    for keyword in keywords:
        if keyword in text:
            return True
    return False

has_keywords_udf = udf(has_keywords)

# add a column to the dataframe that indicates if the article has keywords
df_spark = df_spark.withColumn('has_keywords', has_keywords_udf(df_spark['text_clean']))

# drop articles that do not have keywords
df_spark = df_spark.filter(df_spark['has_keywords'] == True)

# make df_spark the new df_news_final_project
df_news_final_project = df_spark.toPandas()


23/05/12 13:15:28 WARN TaskSetManager: Stage 20 contains a task of very large size (2334 KiB). The maximum recommended task size is 1000 KiB.


In [387]:
# df_news_final_project['text_clean'].apply(lambda x: x.find('apple')).isnull().sum()

,url,date,language,title,text,text_clean,text_notitle,split,split_len,has_keywords
0,https://www.wbko.com/prnewswire/2022/08/25/ult...,2022-08-25,en,UltraSight Receives CE Mark for Novel Cardiac ...,UltraSight Receives CE Mark for Novel Cardiac ...,at 7:30 am cdt|updated : 1 hour agolandmark ac...,"Hahahah 23, 2026\n\nSkip to contentNewsWeather...","[, Hahahah 23, 2026, None, , None, None, Skip ...",16,true
1,https://www.marketscreener.com/quote/stock/POO...,2022-11-08,en,"IN BRIEF: Poolbeg makes ""significant breakthro...","\n\nIN BRIEF: Poolbeg makes ""significant break...",poolbeg pharma plc - london-based clinical sta...,"\n\nHahahah 23, 2026\n \n\n ...","[, None, None, , Hahahah 23, 2026, None, , Non...",745,true
2,https://www.brisbanetimes.com.au/business/smal...,2020-10-27,en,Square Peg aims for the AI sweet spot with lat...,Square Peg aims for the AI sweet spot with lat...,"— 9.00pmsavelog in , register or subscribe to ...","Hahahah 23, 2026Hahahah 23, 2026Skip to sectio...","[, Hahahah 23, 2026, None, , Hahahah 23, 2026,...",13,true
3,https://forextv.com/top-news/forgerock-powers-...,2020-06-23,en,ForgeRock Powers New Era of Digital Identity w...,\n\nForgeRock Powers New Era of Digital Identi...,"( globe newswire ) — forgerock® , the leading ...","\n\nHahahah 23, 2026\n\n \n\n \n\nBreaking New...","[, None, None, , Hahahah 23, 2026, None, , Non...",295,true
4,https://pantagraph.com/opinion/columnists/moha...,2023-04-14,en,Mohammad Hosseini: Should we bring AI into hos...,\nMohammad Hosseini: Should we bring AI into h...,"recently , two major news stories in the techn...","\nHahahah 23, 2026\n\nSkip to main contentSkip...","[, None, None, , Hahahah 23, 2026, None, , Non...",865,true
...,...,...,...,...,...,...,...,...,...,...
384,https://www.kshb.com/news/national/buzzfeed-to...,2023-01-30,en,BuzzFeed to use artificial intelligence for co...,\nBuzzFeed to use artificial intelligence for ...,file photo shows the entrance to buzzfeed in n...,"\nHahahah 23, 2026\n1 weather alerts\n1 closin...","[, None, None, , Hahahah 23, 2026, None, , Non...",628,true
385,https://www.einpresswire.com/article/614170798...,2023-01-31,en,EMERGE Consortium awarded grant by European...,\n EMERGE Consortium awarded grant by Europea...,project scored first in the eic ’ s pathfinder...,\n EMERGE Consortium awarded grant by Europea...,"[, None, None, EMERGE Consortium awarded gra...",1573,true
386,https://www.kktv.com/prnewswire/2021/10/28/pin...,2021-10-28,en,Pinecone Recognized as a 2021 Gartner® Cool Ve...,Pinecone Recognized as a 2021 Gartner® Cool Ve...,"/prnewswire/ -- pinecone systems inc. , a mach...","Hahahah 23, 2026\n\nSkip to contentNewsWeather...","[, Hahahah 23, 2026, None, , None, None, Skip ...",19,true
387,https://www.live5news.com/prnewswire/2021/11/2...,2021-11-24,en,DataRobot and Hexaware Collaborate to Help Cus...,DataRobot and Hexaware Collaborate to Help Cus...,"/prnewswire/ -- hexaware , a global informatio...","Hahahah 23, 2026\n\nSkip to contentSC LotteryL...","[, Hahahah 23, 2026, None, , None, None, Skip ...",16,true


Here we have the cleaned text for downstream analysis. Currently there are 587 documents that are not null and have the mentions of those words in Data Science.

## Sentiment Analysis

In [388]:
df_news_final_project.iloc[:150][['title', 'text_clean']].to_csv('sentanalysis150.csv', index=False)

In [389]:
print(df_news_final_project.iloc[149]['text'])

Arize AI Announces SOC 2 Type II Certification

Skip to contentLIVENewsSportsWeatherInterviewsShop LocalNow HiringSearchHomeLIVENews 4 - LIVELatest NewscastsNews 4 SpecialsProgramming ScheduleRequest a Copy of a News StoryNewsAlabamaFloridaGeorgiaNationalAbsolutely AlabamaSEE it --> SEND itWeatherTracking the TropicsInteractive RadarWeather Map RoomClosingsColor The WeatherSportsSports ScoreboardFNF ScoreboardFriday Night FootballCommunityCommunity CalendarWiregrass PhotobookMr. FoodWeekly SegmentsMobility MondayLegal TalkFeaturesSilent Heroes Of The WiregrassLiving 4 LocalContestsAbout UsMeet the WTVY News TeamAdvertise with WTVYDownload Our AppsInterviewsNewsletterJobs at WTVYCBSNBCMeTVThe CWFull Court Press with Greta Van SusterenCircle - Country Music & LifestyleGray DC BureauInvestigate TVPowerNationLatest NewscastsPress ReleasesArize AI Announces SOC 2 Type II CertificationPublished: Apr. 21, 2022 at 10:00 AM CDT|Updated: 58 minutes agoBERKELEY, Calif., April 21, 2022 /PRNewswire